In [3]:
import os

In [4]:
from keras.preprocessing import image

In [5]:
import cv2

In [6]:
categories = ['with_mask', 'without_mask']

In [7]:
data = []
for category in categories:
    path = os.path.join('train', category)
    
    label = categories.index(category)
    
    for file in os.listdir(path):
        img_path = os.path.join(path, file)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224,224))
        
        data.append([img, label])

In [8]:
len(data)

2142

In [9]:
import random

In [10]:
random.shuffle(data)

In [11]:
X=[]
y=[]

for features, label in data:
    X.append(features)
    y.append(label)

In [12]:
import numpy as np

In [13]:
X = np.array(X)
y = np.array(y)

In [14]:
X.shape

(2142, 224, 224, 3)

In [15]:
y.shape

(2142,)

In [16]:
X = X/255

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [19]:
from keras.applications.vgg16 import VGG16

In [20]:
vgg = VGG16()

In [21]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [22]:
 from keras import Sequential

In [23]:
model = Sequential()

In [24]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [26]:
for layer in model.layers:
    layer.trainable = False

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [28]:
from keras.layers import Dense

In [29]:
model.add(Dense(1, activation = 'sigmoid'))

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [31]:
model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [36]:
model.fit(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

Epoch 1/5
54/54 [==============================] - 298s 6s/step - loss: 0.3893 - accuracy: 0.8360 - val_loss: 0.2150 - val_accuracy: 0.9277
Epoch 2/5
54/54 [==============================] - 304s 6s/step - loss: 0.1602 - accuracy: 0.9743 - val_loss: 0.1324 - val_accuracy: 0.9814
Epoch 3/5
54/54 [==============================] - 294s 5s/step - loss: 0.1124 - accuracy: 0.9796 - val_loss: 0.1032 - val_accuracy: 0.9814
Epoch 4/5
54/54 [==============================] - 291s 5s/step - loss: 0.0907 - accuracy: 0.9807 - val_loss: 0.0870 - val_accuracy: 0.9837
Epoch 5/5
54/54 [==============================] - 300s 6s/step - loss: 0.0787 - accuracy: 0.9831 - val_loss: 0.0770 - val_accuracy: 0.9837


In [32]:
def detect_face_mask(img):
    y_pred = model.predict(img.reshape(1, 224, 224, 3))
#     print(y_pred)
    y_pred = y_pred > 0.5    
    return y_pred

In [33]:
def draw_label(img, text, pos, bg_color):
    text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, cv2.FILLED)
    end_x = pos[0] + text_size[0][0] + 2
    end_y = pos[1] + text_size[0][1] - 2
    
    cv2.rectangle(img, pos, (end_x, end_y), bg_color, cv2.FILLED)
    cv2.putText(img, text, pos, cv2.FONT_HERSHEY_SIMPLEX, 1 ,(0,0,0), 1, cv2.LINE_AA)

In [41]:
haar = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [42]:
def detect_face(img):
    coods = haar.detectMultiScale(img)
    return coods

In [52]:
cap = cv2.VideoCapture(1)

In [53]:
while True:
    ret, frame = cap.read()
    img = cv2.resize(frame, (224, 224))
    y_pred = detect_face_mask(img)
    
    coods = detect_face(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
    
    for x,y,w,h in coods:
        cv2.rectangle(frame, (x,y), (x+w,x+h), (255,0,0), 3)
    if y_pred == 0:
        draw_label(frame, 'Mask', (30, 30), (0, 255, 0))
    else:
        draw_label(frame, 'No mask', (30, 30), (0, 0, 255))
    cv2.imshow("window", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

1/1 [==============================] - 0s 211ms/step


In [54]:
cap.release()